In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import VQC
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import wandb
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 14:46:03.992887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 14:46:03.992947: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 14:46:10.500224: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 14:46:10.500299: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 14:46:10.500790: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [11]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.pca = 16
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.feature_map = 'DoubleAngleMap'
args.ansatz_class = 'TTN'

In [12]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Performing PCA on data...
Cumulative sum on train : 77.73330982270407
Cumulative sum on test: 77.73330982270407
Standardizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 4, 4, 1) │ (634, 4, 4, 1) │ (2115, 4, 4, 1) │ (4, 4, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════

In [13]:
data.config()

{'input_dims': (4, 4, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [14]:
model = VQC(data.config(), args)

In [15]:
model.compile()
model.build_graph().summary()

Model: "VQC-DoubleAngleMap-Chen"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4, 4, 1)]         0         
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 112       
                                                                 
Total params: 112
Trainable params: 80
Non-trainable params: 32
_________________________________________________________________


In [16]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
159/188 [========================>.....] - ETA: 1:36 - loss: 1.0030 - q_auc_2: 0.5212 - custom_accuracy: 0.5213

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

